### Friday, November 17, 2023

Been doing some model cleanup on the hfpt_Oct28 container, and want to re-run this just to ensure all is still well ...

Yup! This still works! Nice!

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 20524MiB of VRAM!


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers


# (base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers
# (base) rob@KAUWITB:~$ 


In [1]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 16.7s to load, takes up 20132Mib VRAM

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


In [5]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 22.5s 20524Mib VRAM




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Hello! I'd be happy to help you with that.

To solve this problem, we can use a combination of natural language processing (NLP) and machine learning techniques. Here's an example of how we can approach this problem:

1. First, we need to preprocess the text data. We can use a library like NLTK or spaCy to tokenize the text, remove stop words, and stem the words.
2. Next, we need to extract the key phrases from the text. We can use a technique like Latent Dirichlet Allocation (LDA) to extract the topics from the text.
3. Once we have the key phrases, we can use a machine learning algorithm like Naive Bayes or Logistic Regression to classify the phrases into categories like "AI", "Machine Learning", "Deep Learning", etc.
4. Finally, we can use a library like Plotly to visualize the resu

In [6]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 11.9s 20524Mib VRAM

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Here's one possible solution:
```
def tell_me_about_ai(input_string):
    if "AI" in input_string:
        return "Artificial intelligence (AI) refers to the ability of machines to perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, making decisions, and solving problems."
    else:
        return "I'm not sure what you mean by 'AI'. Could you explain?"
```
This function checks if the string "AI" is present in the input string. If it is, it returns a definition of artificial intelligence. Otherwise, it asks for further clarification.

Example test cases:
```
print(tell_me_about_ai("What is AI?"))
# Output: Artificial intelligence (AI) refers to the ability of machines to perform tasks that typically require human intelligence, 